In [1]:
import requests
import json
import time
from collections import Counter

# Functie om het terreintype op basis van coördinaten te bepalen
def get_terrain_type(latitude, longitude):
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # De exacte locatie van terreininformatie in de response kan variëren
            return data.get('type', 'Onbekend')
        else:
            return "Fout bij het ophalen van gegevens"
    except Exception as e:
        return str(e)

# Functie om de terreintypes voor de activiteiten te bepalen
def determine_terrain_types(file_path, sample_interval=30):
    terrain_types = {}
    try:
        with open(file_path, 'r') as file:
            activities = json.load(file)
            for activity in activities:
                sampled_terrains = []
                for index in range(0, len(activity['latitude']), sample_interval):
                    latitude = activity['latitude'][index]
                    longitude = activity['longitude'][index]
                    terrain = get_terrain_type(latitude, longitude)
                    sampled_terrains.append(terrain)
                    # time.sleep(1)  # Vertraging om te voldoen aan API-limiet
                most_common_terrains = Counter(sampled_terrains).most_common()
                print(most_common_terrains)
                terrain_types[activity['id']] = most_common_terrains
    except Exception as e:
        return str(e)
    return terrain_types

# Bepaal de terreintypes voor de eerste 100 activiteiten
file_path = '../Data/first_100_activities.json'
terrain_types = determine_terrain_types(file_path)

# Print de meest voorkomende terreintypes per activiteit
if terrain_types:
    for activity_id, common_terrains in terrain_types.items():
        print(f"Activiteit {activity_id}: Meest voorkomende terreintypes: {common_terrains}")
else:
    print("Geen terreintypes bepaald")

[('yes', 7), ('tertiary', 4), ('secondary', 3), ('semidetached_house', 1), ('detached', 1), ('construction', 1)]
[('secondary', 5), ('yes', 5), ('tertiary', 4), ('cycleway', 2), ('semidetached_house', 1)]
[('yes', 6), ('secondary', 5), ('semidetached_house', 1), ('detached', 1), ('tertiary', 1), ('cycleway', 1), ('construction', 1), ('bridge', 1)]
[('secondary', 4), ('tertiary', 4), ('yes', 4), ('house', 3), ('semidetached_house', 1), ('bus_stop', 1)]
[('secondary', 8), ('cycleway', 3), ('apartments', 2), ('semidetached_house', 1), ('terrace', 1), ('duplex', 1), ('construction', 1)]
[('secondary', 9), ('cycleway', 2), ('house', 2), ('civic', 1), ('residential', 1), ('playground', 1), ('bus_stop', 1)]
[('secondary', 9), ('yes', 2), ('residential', 2), ('semidetached_house', 1), ('apartments', 1), ('mall', 1), ('car', 1)]
[('secondary', 8), ('civic', 1), ('terrace', 1), ('construction', 1), ('bridge', 1), ('cycleway', 1), ('parking', 1), ('bus_stop', 1), ('house', 1), ('residential', 1)]

In [2]:
import requests
import json
import time
from collections import Counter

# Functie om het terreintype op basis van coördinaten te bepalen
def get_terrain_type_overpass(latitude, longitude):
    # Query om de dichtstbijzijnde weg/ondergrond te vinden op basis van coördinaten
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
      way(around:50,{latitude},{longitude})["highway"];
      way(around:50,{latitude},{longitude})["landuse"];
      way(around:50,{latitude},{longitude})["natural"];
    );
    out geom;
    """
    try:
        response = requests.get(overpass_url, params={'data': overpass_query})
        if response.status_code == 200:
            data = response.json()
            if data.get('elements'):
                # We nemen het type van het eerste gevonden element
                return data['elements'][0].get('tags').get('highway', 'Onbekend')
            else:
                return "Geen gegevens gevonden"
        else:
            return "Fout bij het ophalen van gegevens"
    except Exception as e:
        return str(e)

# Functie om de terreintypes voor de activiteiten te bepalen
def determine_terrain_types(file_path, sample_interval=30):
    terrain_types = {}
    try:
        with open(file_path, 'r') as file:
            activities = json.load(file)
            for activity in activities:
                sampled_terrains = []
                for index in range(0, len(activity['latitude']), sample_interval):
                    latitude = activity['latitude'][index]
                    longitude = activity['longitude'][index]
                    terrain = get_terrain_type_overpass(latitude, longitude)
                    sampled_terrains.append(terrain)
                most_common_terrains = Counter(sampled_terrains).most_common()
                terrain_types[activity['id']] = most_common_terrains
    except Exception as e:
        print(f"Er is een fout opgetreden: {e}")
        return {}  # Retourneert een lege dictionary in geval van een fout
    return terrain_types


# Bepaal de terreintypes voor de eerste 100 activiteiten
file_path = '../Data/first_100_activities.json'
terrain_types = determine_terrain_types(file_path)

# Print de meest voorkomende terreintypes per activiteit
if terrain_types:
    for activity_id, common_terrains in terrain_types.items():
        print(f"Activiteit {activity_id}: Meest voorkomende terreintypes: {common_terrains}")
else:
    print("Geen terreintypes bepaald")

Activiteit 396826535: Meest voorkomende terreintypes: [('tertiary', 6), ('Onbekend', 4), ('cycleway', 4), ('secondary', 1), ('track', 1), ('construction', 1)]
Activiteit 392337038: Meest voorkomende terreintypes: [('tertiary', 6), ('Onbekend', 3), ('cycleway', 3), ('secondary', 3), ('residential', 1), ('track', 1)]
Activiteit 389643739: Meest voorkomende terreintypes: [('cycleway', 5), ('secondary', 4), ('tertiary', 4), ('Onbekend', 2), ('service', 1), ('residential', 1)]
Activiteit 386729739: Meest voorkomende terreintypes: [('tertiary', 7), ('Onbekend', 4), ('secondary', 4), ('service', 1), ('residential', 1)]
Activiteit 383186560: Meest voorkomende terreintypes: [('cycleway', 4), ('residential', 4), ('Onbekend', 3), ('secondary', 3), ('tertiary', 1), ('trunk_link', 1), ('construction', 1)]
Activiteit 381331596: Meest voorkomende terreintypes: [('secondary', 6), ('residential', 3), ('cycleway', 3), ('Onbekend', 2), ('service', 1), ('platform', 1), ('tertiary', 1)]
Activiteit 38118522

## andere manieren
- Google Maps Roads API
- Here Technologies
- Mapbox
- OpenStreetMap (OSM) Overpass API
- ArcGIS REST API